In [184]:
import copy
import math
import gdown
import numpy as np
import pandas as pd
from collections import defaultdict
from itertools import chain, combinations

In [185]:
def def_value():
    return 0

In [186]:
def generate_candidate_set1(data):
  count = defaultdict(def_value)
  for row in data.keys():
    for item in data[row]:
      count[tuple([item])] += 1

  for item in list(count.keys()):
    if count[item]/n < support_threshold:
      del count[item]

  dict(count)
  return count

In [187]:
def generate_ordered_itemsets(c1, data):

  freq_items = []
  for item in c1.keys():
    freq_items.append([c1[item], list(item)[0]])
  freq_items.sort(reverse=True)
  freq_items_1 = []
  prev=-1
  grp=[]
  for i in range(len(freq_items)):
    if freq_items[i][0]!=prev:
      grp.sort()
      freq_items_1.extend(grp)
      grp=[]
      grp.append(freq_items[i][1])
      prev = freq_items[i][0]
    else:
      grp.append(freq_items[i][1])
  grp.sort()
  freq_items_1.extend(grp)

  for key in data.keys():
    ordered_items = []
    for val in freq_items_1:
      if val in data[key]:
        ordered_items.append(val)
    
    data[key] = ordered_items

  return data, freq_items_1

In [188]:
def generate_trie(data):
  # print(data)
  trie = []
  for key in data.keys():
    for i in range(len(data[key])):
      if len(trie) == i:
        trie.append({
            data[key][i]:
            { 
                'supports': {
                    key
                },
                'next':[]
            }
        })
      else:
        if data[key][i] not in trie[i].keys():
          trie[i][data[key][i]] = {
              'supports':{
                  key
              },
              'next':[]
          }
        else:
          trie[i][data[key][i]]['supports'].add(key)
      if i!=0 and data[key][i] not in trie[i-1][data[key][i-1]]['next']:
          trie[i-1][data[key][i-1]]['next'].append(data[key][i])
  return trie

In [189]:
def default_val():
  return list()

def recur(trie, i, prev_path, cond_pattern_base, item):
  if i < len(trie):
    path = copy.deepcopy(prev_path)
    path.append(item)
    for next_item in trie[i][item]['next']:
      common = {}
      for j in range(len(path)):
        if len(common)<1:
          common = copy.deepcopy(set(trie[j][path[j]]['supports']))
        else:
          common = common.intersection(set(trie[j][path[j]]['supports']))
      common = common.intersection(set(trie[i+1][next_item]['supports']))
      new_support = len(list(common))
      if new_support == 0:
        continue
      cond_pattern_base[next_item].append([path,new_support])
      recur(trie, i+1, path, cond_pattern_base, next_item)

def generate_cond_pattern_base(trie):
  cond_pattern_base = defaultdict(default_val)
  if len(trie) > 0:
  # print(trie)
    for key in trie[0].keys():
      recur(trie, 0, [], cond_pattern_base, key)
  return cond_pattern_base

In [190]:
def powerset(iterable):
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

def get_rules_itemset(itemset, freq, confidence, data):
  rules = []
  for subset in list(powerset(itemset)):
    subset = set(subset)
    if len(subset)>0 and len(subset) != len(list(itemset)):
      count = 0
      for row in data.keys():
        flag = True
        for item in list(subset):
          if item not in data[row]:
            flag = False
            break
        if flag:
          count += 1
      # print(subset, count)
      if count == 0 or count<freq:
        continue
      cal_conf = freq / count
      if cal_conf >= confidence:
        rules.append([set(subset),set(itemset)-set(subset),cal_conf])
  return rules

In [191]:
def generator(prefix, cond_pattern_base, data, frequent_patterns):
  c1 = generate_candidate_set1(data)
  data, c1_keys = generate_ordered_itemsets(c1,data)
  trie = generate_trie(data)
  cond_pattern_base = generate_cond_pattern_base(trie)

  for item in c1_keys:

    extended_prefix = copy.deepcopy(prefix)
    extended_prefix.extend([item])  
    flag = True
    for i in range(len(frequent_patterns)):
      if set(frequent_patterns[i][0]) == set(extended_prefix):
        flag = False
        break
    if flag:
      frequent_patterns.append([list(set(extended_prefix)),c1[tuple([item])]])

  if len(cond_pattern_base.keys()) < 1:
    return
  
  
  for key in cond_pattern_base.keys():
    prefix.append(key)
    
    data = dict()
    counter = 0
    for itemset in cond_pattern_base[key]:
      for i in range(itemset[1]):
        data['T'+str(counter)] = set(itemset[0])
        counter += 1
   
    generator(prefix, copy.deepcopy(cond_pattern_base), data, frequent_patterns)
  

### Dataset 1 - AllElectronics.xlsx

In [192]:
gdown.download(url='https://drive.google.com/file/d/1W80CsLSTFUt6RhXoL4Bxcr7DNDbtDh0Z/view?usp=sharing', output='dataset1.csv', quiet=False, fuzzy=True)

Downloading...
From: https://drive.google.com/uc?id=1W80CsLSTFUt6RhXoL4Bxcr7DNDbtDh0Z
To: /content/dataset1.csv
100%|██████████| 89.0/89.0 [00:00<00:00, 131kB/s]


'dataset1.csv'

In [193]:
def default_set():
  return set()

df = pd.read_csv('dataset1.csv',header=None)

data = defaultdict(default_set)

for i in range(len(df)):
  row = 'T' + str(i)
  for j in range(len(df.iloc[i])):
    if df.iloc[i][j] is not np.nan:
      data[row].add(df.iloc[i][j])

print(data)

defaultdict(<function default_set at 0x7f01dbdfa4c0>, {'T0': {'I5', 'I1', 'I2'}, 'T1': {'I4', 'I2'}, 'T2': {'I3', 'I2'}, 'T3': {'I4', 'I1', 'I2'}, 'T4': {'I3', 'I1'}, 'T5': {'I3', 'I2'}, 'T6': {'I3', 'I1'}, 'T7': {'I5', 'I3', 'I1', 'I2'}, 'T8': {'I3', 'I1', 'I2'}})


In [194]:
data_copy = copy.deepcopy(data)

n = 9
support_threshold = 2/9
confidence_threshold = 0.7
support_val = 2
confidence_val = 0.7*9

In [195]:
frequent_patterns = []

c1 = generate_candidate_set1(data)

data, c1_keys = generate_ordered_itemsets(c1,data)

trie = generate_trie(data)

for key in c1_keys:
  frequent_patterns.append([[key],c1[tuple([key])]])

cond_pattern_base = generate_cond_pattern_base(trie)

for key in cond_pattern_base.keys():
  data = dict()
  prefix = [key]
  counter = 0
  for itemset in cond_pattern_base[key]:
    for i in range(itemset[1]):
      data['T'+str(counter)] = set(itemset[0])
      counter += 1
  
  generator(prefix, cond_pattern_base, data, frequent_patterns)
  

In [196]:
print('No. of freq itemsets:',len(frequent_patterns))
for i in range(len(frequent_patterns)):
  print(frequent_patterns[i][0],'=>', frequent_patterns[i][1])

No. of freq itemsets: 13
['I2'] => 7
['I1'] => 6
['I3'] => 6
['I4'] => 2
['I5'] => 2
['I1', 'I2'] => 4
['I5', 'I1'] => 2
['I5', 'I2'] => 2
['I5', 'I1', 'I2'] => 2
['I4', 'I2'] => 2
['I3', 'I1'] => 4
['I3', 'I2'] => 4
['I3', 'I1', 'I2'] => 2


In [197]:
print('Frequent itemsets of length 3:')
freq = [frequent_patterns[i] for i in range(len(frequent_patterns)) if len(frequent_patterns[i][0])==3]
for i in range(len(freq)):
  print(freq[i][0],'=>', freq[i][1])

Frequent itemsets of length 3:
['I5', 'I1', 'I2'] => 2
['I3', 'I1', 'I2'] => 2


In [198]:
# print(frequent_patterns)
req_len = 3

association_rules = []
for i in range(len(frequent_patterns)):
  if len(frequent_patterns[i][0])>1 and len(frequent_patterns[i][0])==req_len:
    association_rules.extend(get_rules_itemset(frequent_patterns[i][0], frequent_patterns[i][1], confidence_threshold, data_copy))

print('\nAssociation rules which satisfy threshold confidence')
for rule in association_rules:
  print(rule[0],'=> ',rule[1], rule[2])

print('\nTotal no. of associations:',len(association_rules))


Association rules which satisfy threshold confidence
{'I5'} =>  {'I1', 'I2'} 1.0
{'I5', 'I1'} =>  {'I2'} 1.0
{'I5', 'I2'} =>  {'I1'} 1.0

Total no. of associations: 3


### Dataset 2: Goods dataset

In [199]:
gdown.download(url='https://drive.google.com/file/d/15mfojAjnVzbz2ACwbRO1Q58ZqHekeZCu/view?usp=sharing', output='dataset2.csv', quiet=False, fuzzy=True)

Downloading...
From: https://drive.google.com/uc?id=15mfojAjnVzbz2ACwbRO1Q58ZqHekeZCu
To: /content/dataset2.csv
100%|██████████| 9.96M/9.96M [00:00<00:00, 166MB/s]


'dataset2.csv'

In [200]:
df = pd.read_csv('dataset2.csv')

data = defaultdict(default_set)

for i in range(100):
  row = 'T' + str(i)
  for j in range(len(df.iloc[i])):
    if not math.isnan(df.iloc[i][j]):
      data[row].add(str(int(df.iloc[i][j])))

print(data)

defaultdict(<function default_set at 0x7f01dbdfa4c0>, {'T0': {'32', '31', '30'}, 'T1': {'34', '35', '33'}, 'T2': {'44', '46', '45', '38', '40', '39', '41', '37', '42', '36', '43'}, 'T3': {'47', '48', '39', '38'}, 'T4': {'48', '57', '53', '58', '38', '49', '39', '56', '55', '54', '52', '51', '50'}, 'T5': {'60', '41', '62', '32', '59', '61'}, 'T6': {'48', '39', '3'}, 'T7': {'63', '64', '67', '66', '65', '68'}, 'T8': {'32', '69'}, 'T9': {'48', '72', '70', '71'}, 'T10': {'79', '77', '39', '73', '74', '76', '78', '75'}, 'T11': {'79', '48', '80', '38', '41', '39', '36', '81'}, 'T12': {'83', '84', '82'}, 'T13': {'85', '86', '41', '87', '88'}, 'T14': {'48', '89', '91', '94', '39', '95', '100', '98', '93', '90', '96', '99', '101', '97', '92'}, 'T15': {'48', '89', '38', '39', '36'}, 'T16': {'106', '105', '41', '104', '39', '103', '107', '108', '102'}, 'T17': {'110', '38', '41', '39', '109'}, 'T18': {'116', '112', '111', '113', '39', '117', '114', '115', '118'}, 'T19': {'132', '120', '130', '125'

In [201]:
data_copy = copy.deepcopy(data)

n = 100
support_threshold = 3/100
confidence_threshold = 0.7
support_val = 3
confidence_val = 0.7*100

In [202]:
frequent_patterns = []

c1 = generate_candidate_set1(data)

data, c1_keys = generate_ordered_itemsets(c1,data)

trie = generate_trie(data)

for key in c1_keys:
  frequent_patterns.append([[key],c1[tuple([key])]])

cond_pattern_base = generate_cond_pattern_base(trie)

for key in cond_pattern_base.keys():
  data = dict()
  prefix = [key]
  counter = 0
  for itemset in cond_pattern_base[key]:
    for i in range(itemset[1]):
      data['T'+str(counter)] = set(itemset[0])
      counter += 1
  
  generator(prefix, cond_pattern_base, data, frequent_patterns)
  

In [203]:
print('No. of freq itemsets:',len(frequent_patterns))
for i in range(len(frequent_patterns)):
  print(frequent_patterns[i][0],'=>', frequent_patterns[i][1])

No. of freq itemsets: 76
['39'] => 58
['48'] => 47
['38'] => 27
['41'] => 24
['32'] => 10
['36'] => 8
['65'] => 5
['105'] => 4
['110'] => 4
['152'] => 4
['225'] => 4
['89'] => 4
['147'] => 3
['161'] => 3
['170'] => 3
['179'] => 3
['186'] => 3
['237'] => 3
['242'] => 3
['258'] => 3
['310'] => 3
['340'] => 3
['37'] => 3
['60'] => 3
['79'] => 3
['39', '152'] => 3
['39', '38'] => 20
['48', '38'] => 15
['48', '39', '38'] => 11
['41', '39'] => 16
['48', '41'] => 13
['41', '38'] => 10
['41', '39', '38'] => 6
['48', '41', '38'] => 5
['48', '41', '39', '38'] => 3
['36', '38'] => 8
['36', '39'] => 6
['36', '48'] => 4
['36', '41'] => 3
['36', '39', '38'] => 6
['36', '41', '39', '38'] => 3
['48', '38', '41', '39', '36'] => 4
['37', '38'] => 3
['110', '38'] => 3
['110', '39'] => 3
['110', '41'] => 3
['110', '39', '38'] => 3
['105', '39'] => 4
['105', '38'] => 3
['105', '39', '38'] => 3
['39', '237'] => 3
['39', '242'] => 3
['48', '39'] => 34
['79', '39'] => 3
['170', '38'] => 3
['170', '39'] => 3
[

In [205]:
print('Frequent itemsets of length 4:')
freq = [frequent_patterns[i] for i in range(len(frequent_patterns)) if len(frequent_patterns[i][0])==4]
for i in range(len(freq)):
  print(freq[i][0],'=>', freq[i][1])

Frequent itemsets of length 4:
['48', '41', '39', '38'] => 3
['36', '41', '39', '38'] => 3
['170', '48', '39', '38'] => 3
['32', '48', '39', '38'] => 3
['32', '48', '41', '38'] => 3


In [206]:
# print(frequent_patterns)
req_len = 4

association_rules = []
for i in range(len(frequent_patterns)):
  if len(frequent_patterns[i][0])>1 and len(frequent_patterns[i][0])==req_len:
    # print(frequent_patterns[i][0],frequent_patterns[i][1])
    association_rules.extend(get_rules_itemset(frequent_patterns[i][0], frequent_patterns[i][1], confidence_threshold, data_copy))

print('\nAssociation rules which satisfy threshold confidence')
for rule in association_rules:
  print(rule[0],'=> ',rule[1], rule[2])

print('\nTotal no. of associations:',len(association_rules))


Association rules which satisfy threshold confidence
{'36', '41'} =>  {'39', '38'} 1.0
{'36', '41', '38'} =>  {'39'} 1.0
{'170'} =>  {'48', '39', '38'} 1.0
{'170', '48'} =>  {'39', '38'} 1.0
{'170', '39'} =>  {'48', '38'} 1.0
{'170', '38'} =>  {'48', '39'} 1.0
{'170', '48', '39'} =>  {'38'} 1.0
{'170', '48', '38'} =>  {'39'} 1.0
{'170', '39', '38'} =>  {'48'} 1.0
{'32', '38'} =>  {'48', '39'} 1.0
{'32', '48', '38'} =>  {'39'} 1.0
{'32', '39', '38'} =>  {'48'} 1.0
{'32', '41'} =>  {'48', '38'} 0.75
{'32', '38'} =>  {'48', '41'} 1.0
{'32', '48', '41'} =>  {'38'} 1.0
{'32', '48', '38'} =>  {'41'} 1.0

Total no. of associations: 16
